In [2]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
import pandas as pd

In [3]:
urls = [
    "https://www.hioscar.com/clinical-guidelines/medical",
    "https://www.aetna.com/health-care-professionals/clinical-policy-bulletins/medical-clinical-policy-bulletins/medical-clinical-policy-bulletins-search-results.html",
    "https://www.anthem.com/ca/provider/policies/clinical-guidelines/updates/",
    ]
url = urls[0]

In [35]:
"/".join(url.split("/")[:-2])

'https://www.hioscar.com'

In [4]:
def fetch_html(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensures the request was successful
    return BeautifulSoup(response.text, 'html.parser')

def fetch_pdf(url):
    response = requests.get(url)
    response.raise_for_status()
    with pdfplumber.open(response.content) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    return "\n".join(pages) if pages else None

In [5]:
soup = fetch_html(url)
data = {
    "url": url,
    "title": soup.title.string,
    "text": soup.get_text(),
    "html": soup.prettify(),
}
print(data["text"])
print(data["html"])

Clinical Guidelines | Medical | OscarSkip to main content1-855-672-2788 English Log inOscar homeShop PlansMember ResourcesFor Brokers & ProvidersAbout UsFind a DoctorOscar Clinical Guidelines: MedicalClinical guidelines are developed and adopted to establish evidence-based clinical criteria for utilization management decisions. Oscar may delegate utilization management decisions of certain services to third-party delegates, who may develop and adopt their own clinical criteria.Clinical guidelines are applicable to certain policies. Clinical guidelines are applicable to members enrolled in Medicare Advantage plans only if there are no in-force criteria for the specified service in a Centers for Medicare & Medicaid Services (CMS) national coverage determination (NCD) or local coverage determination (LCD) on the date of a prior authorization request. Coverage of services is subject to the terms, conditions, limitations of a member’s policy and applicable state and federal law. Please refe

In [45]:
# Find all 'li' tags with a specific class and then find 'a' tags within them that link to PDFs
li_tags = soup.find_all('li', class_='ListItem_listItemContainer__34sP-')

In [63]:
policies = []
for li in li_tags:
    href = li.find('a').get('href')
    link = "/".join(url.split("/")[:-2]) + href

    policy = {
        "title": li.find('span').text,
        "link": link,
    }
    policies.append(policy)

In [64]:
policy = policies[0]
policy

{'title': 'Acupuncture (CG013)',
 'link': 'https://www.hioscar.com/medical/cg013'}

In [89]:
doc_soup = fetch_html(policy["link"])
iframes = doc_soup.find_all('iframe')
# keep only the first iframe with src ending in .pdf
pdf_url = "https:" + next((iframe.get('src') for iframe in iframes if iframe.get('src').endswith('.pdf')), None)

'https://assets.ctfassets.net/plyq12u1bv8a/rV94yF13niNgBFNAhPxxu/54191136e83fce8fd4b9bd3106ad0504/CG013_Acupuncture.pdf'

In [95]:
response = requests.get(pdf_url)
response.headers

{'Content-Type': 'application/pdf', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Thu, 30 May 2024 21:27:16 GMT', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, HEAD', 'Access-Control-Max-Age': '86400', 'x-amz-replication-status': 'COMPLETED', 'Last-Modified': 'Wed, 17 Apr 2024 18:49:25 GMT', 'ETag': 'W/"9843fd7e045773f4921cf22c6dca2391"', 'x-amz-server-side-encryption': 'AES256', 'Cache-Control': 'max-age=2592000', 'x-amz-version-id': '1cw7jrAnu3Nu88CPdliWBCIr8ajflXSq', 'Server': 'AmazonS3', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1 d64475c5f9123e31ee554a0c3c292f08.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'SFO53-P4', 'X-Amz-Cf-Id': 'JQRWNh5sGhpgR6eyhs6MiNNzfS-THbZpIMkrKYbrgf0quabRfvb2bA==', 'Age': '5843'}

In [96]:
# Local path where you want to save the PDF
file_path = './CG013_Acupuncture.pdf'

# Download the PDF
response = requests.get(pdf_url)
response.raise_for_status()  # Ensure download was successful

# Save the PDF to a local file
with open(file_path, 'wb') as f:
    f.write(response.content)

print(f"PDF successfully saved to {file_path}")

PDF successfully saved to ./CG013_Acupuncture.pdf


In [97]:
# Open the PDF file
with pdfplumber.open(file_path) as pdf:
    # Iterate over each page in the PDF
    all_text = ''
    for page in pdf.pages:
        # Extract text from each page
        text = page.extract_text()
        all_text += text + '\n'  # Collect text from all pages

# Optionally, print the extracted text
print(all_text)

Clinical Guideline
OscarClinicalGuideline:Acupuncture(CG013,Ver.10)
Acupuncture
Disclaimer
Clinicalguidelinesaredevelopedandadoptedtoestablishevidence-basedclinicalcriteriafor
utilizationmanagementdecisions.Clinicalguidelinesareapplicableaccordingtopolicyandplantype.
ThePlanmaydelegateutilizationmanagementdecisionsofcertainservicestothirdpartieswhomay
developandadopttheirownclinicalcriteria.
Coverageofservicesissubjecttotheterms,conditions,andlimitationsofamember’spolicy,aswell
asapplicablestateandfederallaw.Clinicalguidelinesarealsosubjecttoin-forcecriteriasuchasthe
CentersforMedicare&MedicaidServices(CMS)nationalcoveragedetermination(NCD)orlocal
coveragedetermination(LCD)forMedicareAdvantageplans.Pleaserefertothemember’spolicy
documents(e.g.,Certificate/EvidenceofCoverage,ScheduleofBenefits,PlanFormulary)orcontact
thePlantoconfirmcoverage.
Summary
Acupuncturehasbeenwidelypracticedformanyyearsinvariouspartsoftheworld.Itinvolvesthe
stimulationofspecificbodyareasbypenetratingtheskinwith

In [100]:
import PyPDF2

# Open the PDF file
with open(file_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)

    # Iterate over each page and extract text
    text = []
    for page in reader.pages:
        text.append(page.extract_text())

    # Combine text from all pages into a single string
    full_text = "\n".join(text) if text else None

print(full_text)

ClinicalGuideline
OscarClinicalGuideline:Acupuncture(CG013,Ver.10)
Acupuncture
Disclaimer
Clinicalguidelinesaredevelopedandadoptedtoestablishevidence-basedclinicalcriteriafor
utilizationmanagementdecisions.Clinicalguidelinesareapplicableaccordingtopolicyandplantype.
ThePlanmaydelegateutilizationmanagementdecisionsofcertainservicestothirdpartieswhomay
developandadopttheirownclinicalcriteria.
Coverageofservicesissubjecttotheterms,conditions,andlimitationsofamember’spolicy,aswell
asapplicablestateandfederallaw.Clinicalguidelinesarealsosubjecttoin-forcecriteriasuchasthe
CentersforMedicare&MedicaidServices(CMS)nationalcoveragedetermination(NCD)orlocal
coveragedetermination(LCD)forMedicareAdvantageplans.Pleaserefertothemember’spolicy
documents(e.g.,Certiﬁcate/EvidenceofCoverage,ScheduleofBeneﬁts,PlanFormulary)orcontact
thePlantoconﬁrmcoverage.
Summary
Acupuncturehasbeenwidelypracticedformanyyearsinvariouspartsoftheworld.Itinvolvesthe
stimulationofspeciﬁcbodyareasbypenetratingtheskinwithﬁnene

In [83]:
response = requests.get(policy["link"])
response.headers

{'date': 'Thu, 30 May 2024 22:48:08 GMT', 'content-type': 'text/html; charset=utf-8', 'transfer-encoding': 'chunked', 'set-cookie': 'INGRESSCOOKIE-marketing-site-production.marketing.prod.hioscar.com=1717109289.758.16929.995394|434fc1af9ac7d98719518d2009294aec; Max-Age=900; Path=/; Secure; HttpOnly', 'x-powered-by': 'Next.js', 'etag': 'W/"d19a-cdApIxXk9emQfkcTOLH1NVmp1MA"', 'cache-control': 's-maxage=300, stale-while-revalidate', 'content-encoding': 'gzip', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff'}